In [ ]:
# connectomics imports
import platform
import os

platstring = platform.platform()
if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2022/em-connectomics"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/em-connectomics"
elif ('amzn' in platstring):
    # then on AWS
    data_root = "/data/em-connectomics-data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2022/em-connectomics"
    
# OR if you'd like to override the auto options
# data_root = "./proofread_skeletons/"

In [4]:
# this is the EM specific package for querying the EM data
from caveclient import CAVEclient


In [5]:
# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy as sc
%matplotlib widget

In [6]:
# if this errors, go to the CAVEsetup notebook
datastack_name = 'minnie65_public_v343'
client = CAVEclient(datastack_name)

In [7]:
# query the proofreading info of the cells with extended dendritic proofreading
clean_dend_df = client.materialize.query_table('proofreading_status_public_release',
                                              filter_equal_dict={'status_dendrite':'extended'})

In [8]:
clean_dend_df.head()

,id,valid,pt_supervoxel_id,pt_root_id,valid_id,status_dendrite,status_axon,pt_position
0,4,t,91077153340676495,864691135355207375,864691135355207375,extended,non,"[191424, 209888, 22845]"
1,7,t,91570559116360958,864691136602027473,864691136602027473,extended,non,"[194864, 216128, 22205]"
2,8,t,90444453118345856,864691135866846742,864691135866846742,extended,non,"[186768, 214288, 22811]"
3,9,t,89741246779911895,864691135445832338,864691135445832338,extended,non,"[181312, 217904, 23251]"
4,13,t,91499640750531825,864691135569255558,864691135569255558,extended,non,"[194112, 212064, 23175]"


In [9]:
# query a neuron id from the list
neuron_id  = clean_dend_df.iloc[10].pt_root_id
neuron_id

864691135013176726

In [ ]:
# Lets query this neuron, but now splitting columns and converting to um
post_synapse_df = client.materialize.synapse_query(post_ids = neuron_id,desired_resolution=(1000,1000,1000))

# Lets make a matrix of synapse positions for this neuron
syn_pos_nm = np.vstack(post_synapse_df['ctr_pt_position'].values)
# and plot their positions in x,y along with the soma position as a red circle
fig = plt.figure(figsize=(6,12))
ax1 = fig.add_subplot(2, 1, 1, aspect='equal')
ax2 = fig.add_subplot(2, 1, 2,  aspect='equal')
ax1.scatter(syn_pos_nm[:,0], syn_pos_nm[:,1])

# add the soma position as a red circle
soma_pos_df = clean_dend_df.query(f'pt_root_id=={neuron_id}')
soma_pos = soma_pos_df['pt_position'].values
soma_pos = soma_pos[0]
ax1.scatter(soma_pos[0]*4/1000,soma_pos[1]*4/1000,c='r',s=150)
ax1.set_xlabel('x (um)')
ax1.set_ylabel('y (um)')
#ax1.set_ylim(1100,300)

ax2.scatter(syn_pos_nm[:,0], syn_pos_nm[:,2])

# add the soma position as a red circle
ax2.scatter(soma_pos[0]*4/1000,soma_pos[2]*40/1000,c='r',s=150)
ax2.set_xlabel('x (um)')
ax2.set_ylabel('z (um)')
plt.subplots_adjust(hspace=0.01)

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1,1,1, projection='3d')


ax.scatter3D(syn_pos_nm[:,0], syn_pos_nm[:,1], syn_pos_nm[:,2])

# add the soma position as a red circle
ax.scatter3D(soma_pos[0]*4/1000,soma_pos[1]*4/1000,soma_pos[2]*40/1000,c='r',s=500)
plt.subplots_adjust(hspace=0.01)

In [ ]:
neuron_soma_df = client.materialize.query_table('aibs_soma_nuc_metamodel_preds_v117')
neuron_soma_df.head()

In [ ]:
# So a better way to plot a neuron is to use cover_paths
# and plot those as 3d lines
def plot_neuron_skeleton(neuron, ax, c='b', linewidth=1):

    for cover_path in neuron.skeleton.cover_paths:
        path_verts = neuron.skeleton.vertices[cover_path,:]
        ax.plot(path_verts[:,0], path_verts[:,1], path_verts[:,2], c=c, linewidth=linewidth)

fig = plt.figure(figsize=(6, 6))
ax = plt.axes(projection='3d')   
ax.scatter3D(nrn.skeleton.vertices[nrn.skeleton.root][0], nrn.skeleton.vertices[nrn.skeleton.root][1], nrn.skeleton.vertices[nrn.skeleton.root][2])
plot_neuron_skeleton(nrn, ax)

In [ ]:
# general radial scaling law for the entire cell
seg_df = nrn.anno.segment_properties.df
# want to make a directed tree and attach segment properties


In [ ]:
nrn.anno.is_axon #gives mesh labels for axons